## Data Analyst P3 - Jasper Alblas

## Introduction

The aim of this project is to investigate an area from OpenStreetMaps, by auditing its XML data. If any problems arise these should be fixed while converting the dataset in seperate csv files, before finally inserting the data into a SQL database.


## Table of Contents

#### 1. Fetching the data from OpenStreetMap
#### 2. Auditing the data in Python
#### 3. Fixing the data & converting to csv
#### 4. Inserting in SQL Database
#### 5. Run queries

## 1. Fetching the data from OpenStreetMap

For this study I have selected the area in the Netherlands which I grew up in. The following map shows the study area in the southwest of the country:

<img src="area.png">

This dataset is downloaded from https://mapzen.com/data/metro-extracts/.
The uncompressed XML file of this area is 380 MBs large, which is fairly large. For this reason I will take a smaller sample while auditing the data to reduce processing time. The following code is used, from the Udacity SQL For Data Analyst Project Details. 

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

OSM_FILE = "middendelfland.osm"
SAMPLE_FILE = "sample.osm"

k = 50 # Parameter: take every k-th top level element
chosenFile = "middendelfland.osm"  # Variable to change when running the final code on the whole dataset

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

This script uses a value of k to set the sample proportion, with higher values meaning that only every Kth value gets selected in the sample. For example, a k value of 10 would lead to a sample with only approximately 10% of all elements. Throughout this project I have gradually decreased k value, to increase the sample size, before running the final code on the whole XML dataset.

## Auditing the data in Python

Now it's time to start auditing the data in Python. To do this I will create functions to loop through and investigate the dataset. If problems are found, further functions are created to deal with cleaning the data. These will be used while converting the data to csv. 

Let's start by having a quick look at the total number of different tags in the sample.

In [2]:
from collections import defaultdict
    
def count_elements(filename):
    
    count = defaultdict(set)
    
    # Since the dataset is large, we will use iterparse instead of using the parse method 
    # (which gets the whole tree at once)
    
    for event, elem in ET.iterparse(filename, events=("start","end")):
        if event == "start":
            #print elem
            tag = elem.tag
            #print elem.tag, elem.attrib
        
            if tag in count:
                count[tag] += 1
            else:
                count[tag] = 1
            elem.clear()
    return count

In [3]:
count_elements(chosenFile)

defaultdict(set,
            {'bounds': 1,
             'member': 18097,
             'nd': 1779717,
             'node': 1415637,
             'osm': 1,
             'relation': 1725,
             'tag': 2262298,
             'way': 221651})

As we can see, the most common XML element is "tag". This is no surpise as this element is used to describe "node", "way" and "relation" elements. All other elements are typical for OpenStreetMap datasets (https://wiki.openstreetmap.org/wiki/Elements), so this does not indicate any problems. In this study I will focus on the "node" and "way" elements, including their underlying elements.

## The node element

Now that we have taken a look at the number of elements, let's have a look at the other type of element that is of interest, nodes. These describe a specific point of space on the earth's surface defined by its latitude and longitude.

A typical node looks like this:

* node {'changeset': '21644925', 'uid': '1892873', 'timestamp': '2014-04-12T12:07:47Z', 'lon': '4.3676075', 'version': '1', 'user': 'opani_BAG', 'lat': '51.9405121', 'id': '2785239869'}

On first sight, the information worth checking out is its coordinates. Do the coordinates fall within an expected range of value within this area in the Netherlands.

In [4]:
def checkForBadCoordinates(file):
    badLocations = []
    for event, element in ET.iterparse(file, events=("start","end")):
        if element.tag == "node":
            if event == "start":
                if 'lat' and 'lon' in element.attrib:
                    lat = float(element.attrib['lat'])
                    lon = float(element.attrib['lon'])
                
                    # http://www.mapsofworld.com/lat_long/netherlands-lat-long.html
                    if not (lat > 51 and lat < 52,5) and (lon > 4 and lon < 5):
                        print "{0}, {1}".format(lat, lon)
                        badLocations.append(value)
            element.clear()
    return badLocations

checkForBadCoordinates(chosenFile)

[]

As can be seen, no values exists outside of the defined expected area of coordinates.
Now let's have a quick look at the number of unique users, with the total amount of entries counted for each.

In [5]:
def uniqueUsers(filename):
    users = defaultdict(int)
    for event, element in ET.iterparse(filename, events=("start","end")):
        if event == "end":
            if 'uid' in element.attrib:
                value = element.attrib['uid']
                users[value] += 1
            element.clear()
    return [(k, users[k]) for k in sorted(users, key=users.get, reverse=True)]

In [6]:
uniqueUsers(chosenFile)

[('1892873', 740109),
 ('1204291', 313019),
 ('195219', 168920),
 ('36080', 48718),
 ('1989776', 38150),
 ('327477', 35587),
 ('579978', 35046),
 ('191979', 34932),
 ('2005379', 33827),
 ('568540', 18801),
 ('2394881', 12024),
 ('476789', 9855),
 ('329877', 9401),
 ('1989843', 9397),
 ('978786', 9115),
 ('553736', 8359),
 ('483075', 7866),
 ('1771198', 6619),
 ('203114', 6512),
 ('99810', 5820),
 ('665748', 5506),
 ('67141', 3948),
 ('352940', 3353),
 ('2800968', 3279),
 ('12946', 3174),
 ('1758654', 3136),
 ('242798', 3127),
 ('161542', 2784),
 ('13136', 2526),
 ('334075', 2483),
 ('590907', 2314),
 ('1131079', 2185),
 ('1131103', 2020),
 ('9176', 1868),
 ('3087703', 1864),
 ('105512', 1751),
 ('497524', 1431),
 ('301557', 1430),
 ('211771', 1314),
 ('294028', 1314),
 ('436419', 1287),
 ('143643', 1281),
 ('1238039', 1233),
 ('607691', 1212),
 ('36790', 1211),
 ('1890760', 1154),
 ('950776', 983),
 ('149298', 977),
 ('3781654', 967),
 ('2195805', 925),
 ('235579', 838),
 ('5211', 817)

As can be seen, there is a relativly large number of users which inserted only a few values into the dataset, while there is a small number of users responsible for a great amount of data entered. Again, nothing seems out of the ordinary here.

## The way element

Now, let's have a look at the "way" elements. A typical element looks like this: 

* way {'changeset': '27415982', 'uid': '2394881', 'timestamp': '2014-12-12T08:00:03Z', 'version': '3', 'user': "It's so funny_mechanical", 'id': '138652513'}

In my opinion, there is nothing really of interest here for auditing. Even if there were errors here in the timestamp or user ID values, it would not mean much for the remainder of this report.

## The nd element

"nd" elements are nested within "way" elements, to identify which "node" elements are a part of a "way, e.g. a linear path. 

* nd {'ref': '1520353919'}

We could theoratically check if the node this "ref" value is referring to exists in the dataset, but this would be demanding and there is no guarantee that all nodes of the "way" exist in this dataset. I will therefore move on to the "tag" element, which covers the most information.

## The tag element

This element provides the most information of all elements. It is nested within "node", "way", and "relation" elements and describe the meaning of an element to which they are attached. The following are examples of some tags nested within a "node" element:

tag {'k': 'source', 'v': 'BAG 04-10-2013'}

tag {'k': 'addr:city', 'v': 'De Lier'}

tag {'k': 'addr:street', 'v': 'Hohorst'}

tag {'k': 'addr:country', 'v': 'NL'}

tag {'k': 'addr:postcode', 'v': '2678CC'}

tag {'k': 'addr:province', 'v': 'Zuid-Holland'}

tag {'k': 'addr:housenumber', 'v': '37'}

tag {'k': 'bag:gebruiksfunctie', 'v': 'woonfunctie'}

It would be interesting to see which tag keys are the most common.

In [7]:
def uniqueTagKeys(osmfile):
    osm_file = open(osmfile, "r")
    keyValues = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                keyValues[tag.attrib["k"]] += 1
                
    osm_file.close()
    return [(k, keyValues[k]) for k in sorted(keyValues, key=keyValues.get, reverse=True)]


uniqueTagKeys(chosenFile)

[('source', 385338),
 ('source:date', 362864),
 ('addr:city', 202883),
 ('addr:street', 202600),
 ('addr:postcode', 201113),
 ('addr:housenumber', 201096),
 ('building', 165372),
 ('ref:bag', 164128),
 ('start_date', 163308),
 ('highway', 31248),
 ('name', 21216),
 ('building:levels', 11088),
 ('landuse', 10686),
 ('maxspeed', 7286),
 ('roof:colour', 7209),
 ('oneway', 5868),
 ('natural', 4358),
 ('surface', 3768),
 ('bicycle', 3480),
 ('height', 3279),
 ('amenity', 3092),
 ('addr:country', 2584),
 ('addr:province', 2356),
 ('layer', 2181),
 ('construction', 2151),
 ('bag:gebruiksfunctie', 2077),
 ('width', 2007),
 ('bridge', 1851),
 ('foot', 1771),
 ('waterway', 1740),
 ('lanes', 1711),
 ('barrier', 1568),
 ('man_made', 1410),
 ('moped', 1321),
 ('access', 1266),
 ('leisure', 1250),
 ('service', 1246),
 ('door', 1069),
 ('buildingpart', 1065),
 ('ref', 1024),
 ('source:maxspeed', 897),
 ('lit', 885),
 ('railway', 877),
 ('shop', 868),
 ('cycleway', 858),
 ('operator', 802),
 ('traffic

I will now have a closer look to some of the most common keys, starting with source. If problems are found, a function will be created to fix the errors.

### Auditing source

I will now have a look at the data sources. I will create a function which I can reuse for further auditing which lists a count of all unique values.

In [8]:
def uniqueTagValues(osmfile, key):
    osm_file = open(osmfile, "r")
    tagValues = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib["k"] == key:
                    tagValues[tag.attrib["v"]] += 1
    osm_file.close()
    return [(k, tagValues[k]) for k in sorted(tagValues, key=tagValues.get, reverse=True)]

In [9]:
uniqueSources = uniqueTagValues(chosenFile, "source")

uniqueSources

[('BAG', 365937),
 ('3dShapes', 12991),
 ('survey', 1978),
 ('Bing', 1695),
 ('local_knowledge', 837),
 ('BAG 04-10-2013', 371),
 ('Veolia Transport', 319),
 ('AND', 134),
 ('3dShapes;BAG', 130),
 ('Wiki.ovinnederland.nl', 74),
 ('HTM', 57),
 ('BAG;survey', 56),
 ('local knowledge', 49),
 ('Kadaster', 46),
 ('BAG and survey', 38),
 ('BAG;www.reliwiki.nl', 26),
 ('Gemeente Schiedam', 23),
 ('Yahoo imagery good quality', 20),
 ('gps', 19),
 ('Gemeente Westland', 18),
 ('www.zuid-holland.nl', 18),
 ('extrapolation', 18),
 ('Bing;3dShapes', 17),
 ('Rijkswaterstaat.nl', 15),
 ('survey;Bing', 14),
 ('160205_DNZ_002a_markeringen_vast', 14),
 ('survey and bing aerial', 13),
 ('survey 2014-01-16', 12),
 ('Google_maps', 12),
 ('bing aerial', 12),
 ('bing + memory', 12),
 ('Connexxion', 11),
 ('wikipedia', 11),
 ('CBS Bestand Bodemgebruik 2008', 11),
 ('Yahoo', 11),
 ('yahoo_wms', 10),
 ('survey + satelite picture', 10),
 ('BAG + survey', 10),
 ('Bing Sat images', 10),
 ('bing', 10),
 ('BAG, loca

As can be seen, some source values include the actual source, followed by a date. I will try and fix this by removing this date from the source, and adding it to the "source:date" key instead, including transforming to the proper format. If this key already exists I will not change anything, while I will create this value if this is not the case.

In [10]:
from datetime import datetime
import re

def checkForDateAndReturnValues(sourceValue):
    result = re.search('(\d{2})[/.-](\d{2})[/.-](\d{4})$', sourceValue)
    if result:
        # Return the value matched to add it to source:date
        date = result.groups(0)
        
        oldString = "{2}-{1}-{0}".format(date[2],date[1],date[0])
        formattedString = "{2}-{1}-{0}".format(date[0],date[1],date[2])
        
        # Remove from source
        fixedLocation = sourceValue.replace(oldString, "")
        
        return (fixedLocation, formattedString)
    else:
        return None

# Do a quick test
for key,value in uniqueSources:
    values = checkForDateAndReturnValues(key)
    if values: 
        print values
        

('BAG ', '2013-10-04')
('Bing/WorldView-2,DigitalGlobe,Nextview,04/18/2011', '2011-18-04')


This function will be used while transforming the data to csv later on.

### Auditing source date

I will now look at the source dates. A quick function will be created to see if there are any tag values which do not follow the proper format (YYYY-MM-DD).

In [11]:
uniqueDates = uniqueTagValues(chosenFile, "source:date")

uniqueDates

[('2014-02-11', 149089),
 ('2013-11-26', 101856),
 ('2014-03-24', 87969),
 ('2013-10-04', 4353),
 ('2014-08-19', 3877),
 ('2014-08-18', 2361),
 ('2016-06-26', 1688),
 ('2014-08-25', 1654),
 ('2016-01-02', 1546),
 ('2016-06-27', 1037),
 ('2016-07-24', 942),
 ('2016-07-14', 671),
 ('2016-03-20', 633),
 ('2016-10-13', 595),
 ('2015-07-18', 518),
 ('2016-01-04', 514),
 ('2016-10-11', 485),
 ('2016-07-27', 385),
 ('2016-06-23', 316),
 ('2016-08-09', 265),
 ('2014-07-20', 233),
 ('2016-08-14', 180),
 ('2016-08-03', 179),
 ('2016-07-03', 176),
 ('2016-03-17', 175),
 ('2016-08-11', 148),
 ('2016-09-20', 148),
 ('2016-09-25', 112),
 ('2016-07-19', 102),
 ('2016-07-31', 95),
 ('2016-06-18', 92),
 ('2016-05-10', 78),
 ('2014-02-05', 72),
 ('2016-10-12', 66),
 ('2016-02-09', 59),
 ('2016-10-02', 57),
 ('2015-04-01', 41),
 ('2016-01-24', 30),
 ('2016-02-28', 9),
 ('2016-05-12', 7),
 ('2016-09-08', 7),
 ('2016-10-15', 6),
 ('2015-05-25', 4),
 ('2016-10-16', 4),
 ('2015-04-21', 4),
 ('2015-05-17', 3)

In [12]:
for key,count in uniqueDates:
    result = re.match('(\d{4})[/.-](\d{2})[/.-](\d{2})$', key)
    if not result:
        print "Problems: {0}".format(key)

Problems: 2013-11-26 + 2014-02-19
Problems: 2013-09-01;2013-11-26
Problems: 2013-11-26 + local knowledge
Problems: 2013-11-26;survey
Problems: 2013-11-26,2014-10-29


There are only a few problems with the values (with bigger sample sizes!), with either multiple dates added or the source added to the value (actually the opposite issue of last section!). In case there are multiple dates, I will remove the oldest value, since I feel listing the date of the last update makes the most sense. If there is other data I will remove it. I could have added the string to the source key like in the previous section, but it will be hard to check if the remainder actually is a source or something else. 

In [13]:
def cleanSourceDateElement(sourceValue):
    twoDates = re.search('((\d{4}[/.-]\d{2}[/.-]\d{2})[,;](\d{4}[/.-]\d{2}[/.-]\d{2}))$', sourceValue)
    oneDate = re.search('(\d{4}[/.-]\d{2}[/.-]\d{2})$', sourceValue)
    
    if twoDates:
        # Return the value matched to add it to source:date  
        date = twoDates.groups(0)
        dt1 = datetime.strptime(date[1],'%Y-%m-%d')
        dt2 = datetime.strptime(date[2],'%Y-%m-%d')
        
        if dt1 > dt2:
            return date[1]
        else: 
            return date[2]
    elif oneDate:
        # Check for only one date
        date = result.groups(0)
        return "{0}-{1}-{2}".format(date[0],date[1],date[2])
    else:
        return None

# Do a quick test
for key,value in uniqueDates:
    date = cleanSourceDateElement(key)
    if date: 
        print date

2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2013-11-26
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-30
2014-10-29
2014-10-30
2014-10-30


### Auditing country code

Although I don't expect any issues here let's quickly look the "addr:country" key values.

In [14]:
uniqueTagValues(chosenFile, "addr:country")

[('NL', 2584)]

As expected, no problems here.

### Auditing city names

Continuing, I will make sure that all city names actually exist.

In [15]:
# Check all unique city names
uniqueCities = uniqueTagValues(chosenFile, 'addr:city')

uniqueCities

[('Delft', 55206),
 ('Vlaardingen', 38917),
 ('Schiedam', 36215),
 ('Maassluis', 16983),
 ('Naaldwijk', 7521),
 ('Wateringen', 6442),
 ('De Lier', 5850),
 ('Rozenburg', 5048),
 ("'s-Gravenhage", 4098),
 ('Rijswijk', 3786),
 ('Den Hoorn', 3591),
 ('Delfgauw', 3470),
 ('Honselersdijk', 3296),
 ('Maasland', 3130),
 ('Poeldijk', 2850),
 ('Maasdijk', 2078),
 ('Schipluiden', 2051),
 ('Kwintsheul', 1668),
 ('Rotterdam', 571),
 ("'s-Gravenzande", 69),
 ('Pijnacker', 18),
 ('Brielle', 11),
 ('Europoort Rotterdam', 8),
 ('Hoek van Holland', 6)]

To check validity, I will check each city name with a JSON file containing worldwide city names by county, found here: 

https://github.com/David-Haim/CountriesToCitiesJSON/blob/master/countriesToCities.json

In [16]:
import json

def checkCityExistence(name):
    
    json_file='countriesToCities.json'

    json_data = open(json_file)
    data = json.load(json_data)
    json_data.close()

    dutchCities = data['Netherlands']
    
    if name in dutchCities:
        return True
    else:
        return False

In [17]:
for city,count in uniqueCities:
    if not checkCityExistence(city):
        print "Error found with city {0}".format(city)

Error found with city 's-Gravenhage
Error found with city Europoort Rotterdam


's-Gravenhage is actually a synonym for the name "Den Haag" so there is nothing wrong with the value. Europoort Rotterdam should technically perhaps be called Europoort, but this could be argued otherwise. These issues are not required to be changed in my opinion.

### Auditing postal codes

Postal codes are another tag that can be easily checked programatically. For this I will use a fairly forward regular expression. Since dutch postal codes have the format: 0000AA (where 0 is any number, and A is any letter), this can be checked for by the following:

In [18]:
uniqueZip = uniqueTagValues(chosenFile, "addr:postcode")

In [19]:
reZip = re.compile(r'(\d{4})([A-Z]{2})')

for key,value in uniqueZip:
    if reZip.search(key) == None:
        print "Error with zip code {0}".format(key)

Error with zip code 2678 CL
Error with zip code 2678 HA
Error with zip code 2678 AC
Error with zip code 2678 CE
Error with zip code 2678 CH
Error with zip code 2678 MC
Error with zip code 2611 HM
Error with zip code 2678 CG
Error with zip code 2678 CK
Error with zip code 2678 LH
Error with zip code 2678 LN
Error with zip code 2678 CW
Error with zip code 2678 MH
Error with zip code 2678 ST
Error with zip code 2678 AG
Error with zip code 2678 CJ
Error with zip code 2611 BD
Error with zip code 2629 HD
Error with zip code 2678 AW
Error with zip code 2678 AT
Error with zip code 2613 TX
Error with zip code 2678 PT
Error with zip code 2678 EP
Error with zip code 2678 EX
Error with zip code 2678 GC
Error with zip code 2678 CX
Error with zip code 2678 MA
Error with zip code 2611 GP
Error with zip code 2678 EC
Error with zip code 2678 KB
Error with zip code 2678 SV
Error with zip code 2611 BA
Error with zip code 2678 DC
Error with zip code 2622 EE
Error with zip code 3121 KK
Error with zip code 

As can be been, the wrong values are caused by a space between the group of numbers and the two letters. This is a quite common way of writing zip codes in the Netherlands, but it should be changed in the dataset for consistency sake. Since this is the only type of error, I can easily just remove all whitespace by using the python function .replace()

In [20]:
def removeWhiteSpace(element):
    return element.replace(" ", "")

#To test if it works
for key,value in uniqueZip:
    if reZip.search(key) == None:
        print key
        key = removeWhiteSpace(key)
        print key
        break

2678 CL
2678CL


### Auditing street names

I will now have a look at the most common streetnames.

In [21]:
streetValues = uniqueTagValues(chosenFile, "addr:street")
streetValues

[('Merellaan', 872),
 ('Roland Holstlaan', 867),
 ('Rotterdamseweg', 818),
 ('Van Hasseltlaan', 687),
 ('Mozartlaan', 678),
 ('Burgemeester Van Haarenlaan', 651),
 ('Aart van der Leeuwlaan', 642),
 ('Bachplein', 641),
 ('Laan van Wateringse Veld', 610),
 ('Korvezeestraat', 602),
 ('Balthasar van der Polweg', 583),
 ('Zwaluwenlaan', 561),
 (u'R\xf6ntgenweg', 521),
 ('van Hogendorplaan', 521),
 ('Vlaardingerdijk', 503),
 ('Martinus Nijhofflaan', 499),
 ('Aalscholverlaan', 483),
 ('Willem de Zwijgerlaan', 481),
 ('Diepenbrockstraat', 480),
 ('Hoogstraat', 476),
 ('Parkweg', 470),
 ('Rotterdamsedijk', 457),
 ('Schubertlaan', 446),
 ('Koninginnelaan', 444),
 ('Dillenburgsingel', 443),
 ('Westvest', 440),
 ('Poptahof Noord', 438),
 ('E. du Perronlaan', 428),
 ('Hoofdstraat', 406),
 ('Papsouwselaan', 402),
 ('Schiedamseweg', 399),
 ('Frank van Borselenstraat', 387),
 (u'H\xe4ndellaan', 386),
 ('Holysingel', 384),
 ('Oude Delft', 371),
 (u'Petronella Vo\xfbtestraat', 369),
 ('Sperwerlaan', 366

I created a list of common Dutch streetname endings. The problem is, that unlike U.S. streetnames, the words are connected so it will be diifcult to use a regular expression to seperate the street type from the rest of the name. This will thus required more manual labor. Let's have a look to see which values do not include any of the common Dutch streetname words.

In [22]:
dutchStreetNames = ["straat", "dijk", "plein", "kade", "gracht", "laan", "sluis", "polder", "hof", "weg", \
                    "gaag", "singel", "dreef", "pad", "gaag", "vliet", "hoeve"]

for key,value in streetValues:
    if not any(street in key for street in dutchStreetNames):
        print key


Laan van Wateringse Veld
Westvest
Oude Delft
Pijperring
Sparrendal
Buitenwatersloot
Singel
De Vloot
Oostblok
Robert Schumanring
Nieuwe Haven
Oosteinde
Laan der Zeven Linden
Warande
Lange Haven
Broersvest
Tjalk
Raam
Markt
Westlander
Meester Beerninkplantsoen
Drogerij
Rozemarijn
Westplantsoen
Parkzoom
Nagelkruid
Honderdland
Koornmarkt
Sprinterplaats
Windmolen
Gedempte Biersloot
Liesveld
Brabantse Turfmarkt
De Velden
Vondelpark
Burgwal
Dennendal
Schuttersveld
Hofzicht
Uitterlier
Maasboulevard
De Vlinderhoven
Over de Vesten
Oliemolen
Achterom
Witmolen
Elzendal
Duizendguldenkruid
Broersveld
Rietveld
Essendaal
Burgemeestersrand
Lenteblok
Koekamp
Olmendal
Wittebrug
Wipperspark
Het Tolland
Noordeinde
Fluitekruid
Touwbaan
Bleijenburg
De Kringloop
Bizet
Leeuwerik
Boumare
Laan van Bol'Es
Grutto
Grote Stern
Barbarasteeg
Steenen Dijck
Ververij
Korte Haven
Goudappel
Billitonflat
Oude IJsbaan
Drevenpark
Gerstakker
Straat van Ormoes
Buis
Zuideinde
Palmyraplaats
Molenweide
Bellefleur
Oost-Indiëplaats
K

As can be seen, even with excluding some of the most common Dutch streetname endings, there are still many streetname which do not include any of these. A quick overview does not show any clearly wrong streetname so I will leave it at that.


### Auditing landuse

As someone with a geography background, I wanted to have a closer look at the tags related to landuse.

In [37]:
uniqueLandUses = uniqueTagValues(chosenFile, 'landuse')

uniqueLandUses[:10]

[('grass', 5763),
 ('forest', 3990),
 ('farmyard', 143),
 ('construction', 94),
 ('residential', 91),
 ('farm', 88),
 ('industrial', 83),
 ('meadow', 71),
 ('reservoir', 62),
 ('cemetery', 54)]

These values look fine with proper names in English.

### Auditing problems in tag keys

Finally, I will have a quick look at the tag keys to see if any of these have problematic characters.

In [24]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_|:)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    
def countKeyType(element):
    if element.tag == "tag":
        if element.attrib["k"]:
            value = element.attrib["k"]
            if problemchars.search(value) != None:
                keys["problemchars"] += 1
            elif lower.search(value) != None:
                keys["lower"] += 1
            elif lower_colon.search(value) != None:
                keys["lower_colon"] += 1
            else:
                print value
                keys["other"] += 1        
    return keys

def process_map(filename):

    for _, element in ET.iterparse(filename):
        keys = countKeyType(element)

    return keys

process_map(chosenFile)


fuel:octane_95
fuel:octane_95
note1
FIXME
fuel:e10
fuel:e85
fuel:1_25
fuel:1_50
fuel:octane_91
fuel:octane_95
fuel:octane_98
fuel:GTL_diesel
fuel:HGV_diesel
fuel:octane_100
seamark:light:1:range
seamark:light:1:colour
seamark:light:1:height
seamark:light:1:category
seamark:light:1:character
seamark:light:1:sector_end
seamark:light:1:sector_start
fuel:octane_95
source2
fuel:e10
fuel:e85
fuel:1_25
fuel:1_50
fuel:octane_91
fuel:octane_95
fuel:octane_98
fuel:GTL_diesel
fuel:HGV_diesel
fuel:octane_100
seamark:bridge:clearance_width_1
socket:type2
socket:type2
currency:EUR
source_1
source_2
payment:ov-chipcard
fuel:octane_95
route_ref:De_Lijn
url:m3u8
seamark:light:1:colour
seamark:light:1:height
seamark:light:1:period
seamark:light:1:character
seamark:light:1:sector_end
seamark:light:1:sector_start
currency:EUR
railway:atb-eg
railway:atb-ng
railway:atb-eg
railway:atb-ng
railway:atb-eg
railway:atb-ng
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
CEMT
FIXME
3dshap

{'lower': 877102, 'lower_colon': 1385065, 'other': 131, 'problemchars': 0}

This shows that there are no keys with problematic chars, which is the most important since these could cause problems while inserting to SQL. 

The "lower" and "lower_colon" results will be used in the next section, where I will convert the XML to multiple csv files. The differently types of keys will be treated differently while problematic keys will be removed.

The "other" values seem to be caused by the presence of uppercase letters, underscores, and '-'. I do not expect these to cause errors and I am not aware of any rules against these characters, so I will not fix these values.

I will create a quick function that can be used while converting the XML to csv.

In [25]:
from enum import Enum

class KeyType(Enum):
    ProblemChars = 1
    Lower = 2
    LowerColon = 3
    Other = 4

def returnKeyType(value):
    if problemchars.search(value) != None:
        return KeyType.ProblemChars
    elif lower.search(value) != None:
        return KeyType.Lower
    elif lower_colon.search(value) != None:
        return KeyType.LowerColon
    else:
        return KeyType.Other      

## Fixing the data and exporting to CSV

Now I will convert the XML into csv. I will at the same time fix several issues which I have investigated in the previous chapter. These are: source keys, source date, postal codes, as well as problems in tag keys. For these I will use the earlier created functions:

- checkForDateAndReturnValues(sourceValue)
- cleanSourceDateElement(sourceValue)
- removeWhitespace(element)
- returnKeyType(key)

In [26]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == "node":
        for value in node_attr_fields:
            if value in element.attrib:
                node_attribs[value] = element.attrib[value]
        
        # Check tags
        for child in element.iter('tag'):
            keyValue = child.attrib['k']
            value = child.attrib['v']
            
            # If key value is source
            if keyValue == "source":
                values = checkForDateAndReturnValues(child.attrib['v'])
                if values:
                    value = values[0]
                    newDate = values[1]
                    
                    # If new date, find proper tag and set
                    for siblingTag in element.iter('tag'):
                        keyValue = siblingTag.attrib['k']
                        if keyValue == "source:date":
                            siblingTag.attrib['v'] = newDate
                
            # If key value is source date
            if keyValue == "source:date":
                cleanedSource = cleanSourceDateElement(value)
                if cleanedSource:
                    value = cleanedSource
            
            # If key value is postal code
            if keyValue == "addr:postcode":
                cleanedPostal = removeWhiteSpace(value)
                if cleanedPostal:
                    value = cleanedPostal
            
            keyType = returnKeyType(keyValue)
            
            if keyType == KeyType.ProblemChars:
                continue
            
            seperateValues = keyValue.split(':',1)
            typeValue = ""

            if keyType == KeyType.LowerColon:
                typeValue = seperateValues[0]
                keyValue = seperateValues[1]
            else:
                typeValue="regular"
            tag = {'id': element.attrib['id'], 'key': keyValue, 'value': value, 'type': typeValue}
            tags.append(tag)
        
    elif element.tag == 'way':
        for value in way_attr_fields:
            if value in element.attrib:
                way_attribs[value] = element.attrib[value]
        
        # Check nd 
        count = 0
        for child in element.iter('nd'):
            ref = child.attrib['ref']
            tag = {'id': element.attrib['id'], 'node_id': ref, 'position': count}
            way_nodes.append(tag)
            count += 1
        
        # Check tags
        for child in element.iter('tag'):
            keyValue = child.attrib['k']
            
            # If key value is source
            if keyValue == "source":
                values = checkForDateAndReturnValues(child.attrib['v'])
                if values:
                    value = values[0]
                    newDate = values[1]
                    
                    # If new date, find proper tag and set
                    for siblingTag in element.iter('tag'):
                        keyValue = siblingTag.attrib['k']
                        if keyValue == "source:date":
                            siblingTag.attrib['v'] = newDate
                
            # If key value is source date
            if keyValue == "source:date":
                cleanedSource = cleanSourceDateElement(value)
                if cleanedSource:
                    value = cleanedSource
            
            # If key value is postal code
            if keyValue == "addr:postcode":
                cleanedPostal = removeWhiteSpace(value)
                if cleanedPostal:
                    value = cleanedPostal
            
            keyType = returnKeyType(keyValue)

            if keyType == KeyType.ProblemChars:
                continue
            
            seperateValues = keyValue.split(':',1)
            typeValue = ""

            if keyType == KeyType.LowerColon:
                typeValue = seperateValues[0]
                keyValue = seperateValues[1]
            else:
                typeValue="regular"
                
            tag = {'id': element.attrib['id'], 'key': keyValue, 'value': child.attrib['v'], 'type': typeValue}
            tags.append(tag)

    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



The following functions are from the SQL For Data Analysis section of the Wrangle OpenstreetMap by Udacity. In short, these help looping through the XML data, converting it to csv, and validate the results. 

In [27]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)


Everything seems fine, so now let's insert the csv files into SQL.

## 5. Inserting in SQL Database

Now that the CSV file are created, we will proceed by adding importing them into SQL. While this can be done through Python, it is quicker to use some SQLite scripts in the command prompt.

In [28]:
# Terminal commands used, with the third line run for each of the five csv files

# 1. sqlite3 elements
# 2. .mode csv
# 3. import nodes.csv
# 4 .tables

## 6. Running queries

With everything setup I can now proceed to run some queries. This could be done in the terminal, but it is easier to show of right here through Python. Let's see if everything works by creating a simple function that simply fetches a number of rows of a specific table and column. Afterwards I will do a few other queries to look at some interesting data.

In [30]:
import sqlite3
from pprint import pprint

# change this to 'sqlite_file =  Chinook_Sqlite.sqlite' if that is the file you have 
sqlite_file = 'elements'    # name of the sqlite database file

def queryDatabaseWithTableAndColumnName(tableName, columnName, limit):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()

    for row in c.execute('SELECT {cn} FROM {tn} LIMIT {l}'.\
            format(tn=tableName, cn=columnName, l=limit)):
        print(row)

    conn.close()

queryDatabaseWithTableAndColumnName('nodes', "user", 10)    

(u'nimapper',)
(u'PTT',)
(u'datalogg',)
(u'CeesW',)
(u'OSMF Redaction Account',)
(u'AnkEric',)
(u'brandmeester',)
(u'AND',)
(u'milovanderlinden',)
(u'aarnout',)


Let's look at the number of nodes and ways elements.

In [31]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('SELECT COUNT(*) FROM nodes')

result = c.fetchall()
print(result)

conn.close()


[(28313,)]


In [32]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('SELECT COUNT(*) FROM ways')

result = c.fetchall()
print(result)

conn.close()

[(4433,)]


Both these values are the same as the count I did in the start in Python, showing that all data came over.

Now let's the top 10 active users

In [33]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('SELECT totalUsers.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) as totalUsers GROUP BY totalUsers.user ORDER BY num DESC LIMIT 10;')

result = c.fetchall()
pprint(result)

conn.close()

[(u'opani_BAG', 14787),
 (u"It's so funny_BAG", 6269),
 (u'3dShapes', 3361),
 (u'nimapper', 979),
 (u'Sander H_BAG', 762),
 (u'Sander H', 718),
 (u"It's so funny", 708),
 (u'Von Humboldt', 705),
 (u'Martin Borsje_BAG', 675),
 (u'Marc-sch', 374)]


Let's find all tags from the most active user.

In [34]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('SELECT count(*) FROM ways,ways_tags WHERE ways.id = ways_tags.id AND ways.user = "opani_BAG";')

result = c.fetchall()
pprint(result)

conn.close()

[(10550,)]


In [35]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('SELECT count(*) FROM nodes,nodes_tags WHERE nodes.id = nodes_tags.id AND nodes.user = "opani_BAG";')

result = c.fetchall()
pprint(result)

conn.close()

[(14654,)]


Number of unique city names:

In [36]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('SELECT cities.value, COUNT(*) as num FROM (SELECT value FROM nodes_tags WHERE type = "addr" AND key = "city"UNION ALL SELECT value FROM ways_tags WHERE type = "addr" AND key = "city") as cities GROUP BY cities.value ORDER BY num DESC LIMIT 10;')

result = c.fetchall()
pprint(result)

conn.close()

[(u'Delft', 1114),
 (u'Vlaardingen', 780),
 (u'Schiedam', 722),
 (u'Maassluis', 340),
 (u'Naaldwijk', 149),
 (u'Wateringen', 131),
 (u'De Lier', 123),
 (u'Rozenburg', 100),
 (u"'s-Gravenhage", 83),
 (u'Rijswijk', 77)]


## Discussion

OpenstreetMaps hosts an enormous amount of data, which is great for data scientists. But the open source nature of the API also leads to some disadvantages. This can clearly be seen in the amount of different tag key values. Since persons are not forced to follow some strict standard concerning value formats, there seem to be a lot of keys with a low frequency. An extreme example are the source key values as seen earlier. There are so many different values, with difference between formatting, as well as persons adding multiple sources.

Another issue I came across was the mix between Dutch and English both in tag keys and tag values. There is for example the land use keys, but with many other elements this key had the dutch equivalent "gebruiksfunctie". While I did not decide to clean this up in this project, it just shows that before being able to properly use this dataset in for data analysis a lot more cleanup needs to be done, with some of these requiring manual labor.

Some of the cleaning methods in this project might not work while analyzing data in other areas in the Netherlands. It could for example be true that zip codes are written in another format which was not taken into account, but the aim in this project was to fix some of the biggest issues in relation to this specific dataset.

## Conclusion

This project has shown how to convert data from different sources in to different formats, and to finally save this data in a SQL database. It has also shown how common errors can be in usergenerated data and the importance of data wrangling. In general I felt that this dataset did not have as many problems as I perhaps anticipiated, but I hope the process of auditing and cleaning data was clear from this project. 